In [1]:
using CSV
using DataFrames
using DataStructures: OrderedDict
using HTTP
using JSON3

include("census_vars.jl");
include("secrets.jl")

ENV["census_key"] = census_key;

In [2]:
function format_ucgid(;agg="tract", state="", comp="Not a geographic component")
    if state != ""
        return "pseudo(04000$(geographic_components[comp])US$(states[state])\$$(geographies[agg])0000)"
    end
    return "pseudo(01000$(geographic_components[comp])US\$$(geographies[agg])0000)"
end

function query_census(surv, var; agg="county", state="")
    if var[1] == 'D'
        url = "https://api.census.gov/data/$surv/profile"
    elseif var[1] == 'S'
        url = "https://api.census.gov/data/$surv/subject"
    else
        url = "https://api.census.gov/data/$surv"
    end
    
    ucgid = format_ucgid(agg=agg, state=state)
    query = Dict([
        ("key", ENV["census_key"]),
        ("get", join(("NAME", var), ",")),
        ("ucgid", ucgid)
    ])

    println(var)
    println(url)
    println(query)
    println()
    
    r = HTTP.get(
        url;
        query = query
    )
    
    body = JSON3.read(r.body)
    header, data = body[1], body[2:end]
    
    df = OrderedDict{Symbol, Vector}()
    for (i, col_name) in enumerate(header)
        df[Symbol(col_name)] = [row[i] for row in data]
    end

    sleep(5)
    
    return DataFrame(df);
end

query_census (generic function with 1 method)

In [3]:
surv_dec = "2020/dec/dhc"
vars_dec = [
    # rurality
    "Rural" => "H2_003N"
    "Urban" => "H2_002N"
    "Urban + Rural" => "H2_001N"
]

surv_acs = "2023/acs/acs5"
vars_acs = [
    "TotalPopulation" => "B01001_001E"
    
    # Single-Parent Households
    "Male led Household with childeren" => "B11005_006E"
    "Female led Household with childeren" => "B11005_007E"
    # Nonfamilty household with child
    "NonFamily Household with childeren" => "B11005_008E"
    "Households with childeren" => "B11005_002E"

    # youth not in school
    "Male 16-19 no HS" => "B14005_012E"
    "Female 16-19 no HS" => "B14005_026E"
    "Female + Male 16-19" => "B14005_001E"

    # Pay Gap Sex
    "Male median earnings" => "B20001_002E"
    "Female median earnings" => "B20001_023E"

    # Pay Gap Race
    "White median earnings" => "B20017A_001E"
    "Black median earnings" => "B20017B_001E"

    # Housing Gap
    "Median House Price" => "B19013_001E"
    "Median Salary" => "B25077_001E"

    "Unemployment Rate" => "S2301_C04_001E"
    "Owner Occupied Housing" => "DP04_0046PE"

    #white collar jobs
    "White collar workers" => "C24060_002E"
    "Total workers" => "C24060_001E"

    # SNAP
    "Total Households" => "B22002_001E"
    "Total Households w/ SNAP" => "B22002_002E"
    "Total Households w/ SNAP + childeren" => "B22002_003E"
    "Total Households w/ SNAP + elders" => "B22001_003E"

    #tech
    "Households w/o Internet" => "B28002_013E"
    "Households w/ Broadband Internet" => "B28002_004E"
    "Households w/ Smartphone" => "B28010_005E"

    # char
    "Households w/o Car" => "DP04_0058E"
    "Households w/o Complete Plumbing" => "DP04_0073PE"
    "Households w/o Complete Kitchen" => "DP04_0074E"
    
    # transport
    "Over 60 min commuting" => "S0801_C01_045E"
    "WFH" => "S0801_C01_013E"
    "Public Transit to Work" => "S0801_C01_009E"
    "Walked to Work" => "S0801_C01_010E"
    "Biked to Work" => "S0801_C01_011E"
    "Total Workers" => "S0801_C01_001E"
];

In [4]:
dfs = [
    [query_census(surv_acs, var, agg="county")[!, ["ucgid", "NAME", var]] for (_, var) in vars_acs];
    [query_census(surv_dec, var, agg="county")[!, ["ucgid", "NAME", var]] for (_, var) in vars_dec]
]

df = dfs[1]
for next_df in dfs[2:end]
    leftjoin!(df, next_df, on=["NAME", "ucgid"])
end

fname = "raw_data/census/census_data_county_raw.csv"
CSV.write(fname, df)

B01001_001E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B01001_001E", "ucgid" => "pseudo(0100000US\$0500000)")

B11005_006E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B11005_006E", "ucgid" => "pseudo(0100000US\$0500000)")

B11005_007E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B11005_007E", "ucgid" => "pseudo(0100000US\$0500000)")

B11005_008E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B11005_008E", "ucgid" => "pseudo(0100000US\$0500000)")

B11005_002E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B11005_002E", "ucgid" => "pseudo(0100000US\$0500000)")

B14005_012E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c

"raw_data/census/census_data_county_raw.csv"

In [5]:
dfs = [
    [query_census(surv_acs, var, agg="zip code tabulation area")[!, ["ucgid", "NAME", var]] for (_, var) in vars_acs];
    [query_census(surv_dec, var, agg="zip code tabulation area")[!, ["ucgid", "NAME", var]] for (_, var) in vars_dec]
]

df = dfs[1]
for next_df in dfs[2:end]
    leftjoin!(df, next_df, on=["NAME", "ucgid"])
end

fname = "raw_data/census/census_data_zcta_raw.csv"
CSV.write(fname, df)

B01001_001E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B01001_001E", "ucgid" => "pseudo(0100000US\$8600000)")

B11005_006E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B11005_006E", "ucgid" => "pseudo(0100000US\$8600000)")

B11005_007E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B11005_007E", "ucgid" => "pseudo(0100000US\$8600000)")

B11005_008E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B11005_008E", "ucgid" => "pseudo(0100000US\$8600000)")

B11005_002E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B11005_002E", "ucgid" => "pseudo(0100000US\$8600000)")

B14005_012E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c

"raw_data/census/census_data_zcta_raw.csv"

In [6]:
dfs = [
    [query_census(surv_acs, var, agg="tract")[!, ["ucgid", "NAME", var]] for (_, var) in vars_acs];
    [query_census(surv_dec, var, agg="tract")[!, ["ucgid", "NAME", var]] for (_, var) in vars_dec]
]

df = dfs[1]
for next_df in dfs[2:end]
    leftjoin!(df, next_df, on=["NAME", "ucgid"])
end

fname = "raw_data/census/census_data_tract_raw.csv"
CSV.write(fname, df)

B01001_001E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B01001_001E", "ucgid" => "pseudo(0100000US\$1400000)")

B11005_006E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B11005_006E", "ucgid" => "pseudo(0100000US\$1400000)")

B11005_007E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B11005_007E", "ucgid" => "pseudo(0100000US\$1400000)")

B11005_008E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B11005_008E", "ucgid" => "pseudo(0100000US\$1400000)")

B11005_002E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B11005_002E", "ucgid" => "pseudo(0100000US\$1400000)")

B14005_012E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c

LoadError: SystemError: opening file "raw_data/census/census_data_tract_raw.csv": Permission denied

In [5]:
surv_acs = "2023/acs/acs5"
vars = [
    ("B19013_001E" => "MedHHInc")
    ("B19054_002E" => "PctRecvIDR_num")
    ("B19054_001E" => "PctRecvIDR_den")
    ("B19058_002E" => "PctPubAsst_num")
    ("B19058_001E" => "PctPubAsst_den")
    ("B25077_001E" => "MedHomeVal")
    ("C24060_002E" => "PctMgmtBusScArti_num")
    ("C24060_001E" => "PctMgmtBusScArti_den")
    ("B11005_007E" => "PctFemHeadKids_num1")
    ("B11005_010E" => "PctFemHeadKids_num2")
    ("B11005_001E" => "PctFemHeadKids_den")
    ("DP04_0046PE" => "PctOwnerOcc")
    ("DP04_0075PE" => "PctNoPhone")
    ("DP04_0073PE" => "PctNComPlmb")
    ("S1501_C01_009E" => "PctEduc_num25upHS")
    ("S1501_C01_010E" => "PctEduc_num25upSC")
    ("S1501_C01_011E" => "PctEduc_num25upAD")
    ("S1501_C01_012E" => "PctEduc_num25upBD")
    ("S1501_C01_013E" => "PctEduc_num25upGD")
    ("S1501_C01_006E" => "PctEduc_den25up")
    ("S1702_C02_001E" => "PctFamBelowPov")
    ("S2301_C04_001E" => "PctUnempl")
    ("B01001_001E" => "TotalPopulation")
]

dfs = [query_census(surv_acs, var, agg="county")[!, ["ucgid", "NAME", var]] for (var, _) in vars]
;

B19013_001E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B19013_001E", "ucgid" => "pseudo(0100000US\$0500000)")

B19054_002E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B19054_002E", "ucgid" => "pseudo(0100000US\$0500000)")

B19054_001E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B19054_001E", "ucgid" => "pseudo(0100000US\$0500000)")

B19058_002E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B19058_002E", "ucgid" => "pseudo(0100000US\$0500000)")

B19058_001E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B19058_001E", "ucgid" => "pseudo(0100000US\$0500000)")

B25077_001E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c

In [16]:
df = dfs[1]
for next_df in dfs[2:end]
    leftjoin!(df, next_df, on=["NAME", "ucgid"])
end

fname = "raw_data/census/census_data_ndi_county_raw.csv"
CSV.write(fname, df)

"raw_data/census/census_data_ndi_county_raw.csv"

In [17]:
fname = "raw_data/census/census_data_ndi_county_raw.csv"
df = DataFrame(CSV.File(fname));

In [18]:
rename!(df, vars);

In [19]:
function fix(s)
    if s in [-666666666, -999999999, -888888888, -222222222, -333333333, -555555555,
             "-666666666", "-999999999", "-888888888", "-222222222", "-333333333", "-555555555",
             "Varies", "*", "null"]
        return NaN
    elseif typeof(s) == String
        return parse(Float64, s)
    else
        return s
    end
end
        
    
for (_, name) in vars
    df[!, name] = [fix(e) for e in df[!, name]]
end

In [20]:
df[!, "PctRecvIDR"] = 100 * df[!, "PctRecvIDR_num"] ./ df[!, "PctRecvIDR_den"]
df[!, "PctPubAsst"] = 100 * df[!, "PctPubAsst_num"] ./ df[!, "PctPubAsst_den"]
df[!, "PctMgmtBusSciArt"] = 100 * df[!, "PctMgmtBusScArti_num"] ./ df[!, "PctMgmtBusScArti_den"]
df[!, "PctFemHeadKids"] = 100 * (df[!, "PctFemHeadKids_num1"] + df[!, "PctFemHeadKids_num2"]) ./ df[!, "PctFemHeadKids_den"]
df[!, "PctEducHSPlus"] = (df[!, "PctEduc_num25upHS"] .+ df[!, "PctEduc_num25upSC"] .+ df[!, "PctEduc_num25upAD"] \
                       .+ df[!, "PctEduc_num25upBD"] .+ df[!, "PctEduc_num25upGD"]) ./ df[!, "PctEduc_den25up"] * 100
df[!, "PctEducBchPlus"] = 100 * (df[!, "PctEduc_num25upBD"] .+ df[!, "PctEduc_num25upGD"]) ./ df[!, "PctEduc_den25up"]

df[!, "PctRecvIDR"] = 100 * df[!, "PctRecvIDR_num"] ./ df[!, "PctRecvIDR_den"]
df[!, "PctPubAsst"] = 100 * df[!, "PctMgmtBusScArti_num"] ./ df[!, "PctMgmtBusScArti_den"]
df[!, "PctFemHeadKids"] = 100 * (df[!, "PctFemHeadKids_num1"] + df[!, "PctFemHeadKids_num2"]) ./ df[!, "PctFemHeadKids_den"]
df[!, "PctEducHSPlus"] = (df[!, "PctEduc_num25upHS"] .+ df[!, "PctEduc_num25upSC"] .+ df[!, "PctEduc_num25upAD"] \
                       .+ df[!, "PctEduc_num25upBD"] .+ df[!, "PctEduc_num25upGD"]) ./ df[!, "PctEduc_den25up"] * 100
df[!, "PctEducBchPlus"] = 100 * (df[!, "PctEduc_num25upBD"] + df[!, "PctEduc_num25upGD"]) ./ df[!, "PctEduc_den25up"]

df[!, "PctNoIDR"] = 100 .- df[!, "PctRecvIDR"]
df[!, "PctWorkClass"] = 100 .- df[!, "PctMgmtBusSciArt"]
df[!, "PctNotOwnerOcc"] = 100 .- df[!, "PctOwnerOcc"]
df[!, "PctEducLTHS"] = 100 .- df[!, "PctEducHSPlus"]
df[!, "PctEducLTBch"] = 100 .- df[!, "PctEducBchPlus"]

#df[!, "logMedHHInc"] = log.(df[!, "MedHHInc"])
#df[!, "logMedHomeVal"] = log.(df[!, "MedHomeVal"])
;

In [21]:
cols = ["ucgid",
        "NAME",
        "MedHHInc",
        "PctNoIDR",
        "PctPubAsst",
        "MedHomeVal",
        "PctWorkClass",
        "PctFemHeadKids",
        "PctNotOwnerOcc",
        "PctNoPhone",
        "PctNComPlmb",
        "PctEducLTHS",
        "PctEducLTBch",
        "PctFamBelowPov",
        "PctUnempl",
        "TotalPopulation"]


fname = "raw_data/census/census_data_ndi_county.csv"
CSV.write(fname, df[!, cols])

"raw_data/census/census_data_ndi_county.csv"

In [31]:
dfs = [query_census(surv_acs, var, agg="tract")[!, ["ucgid", "NAME", var]] for (var, _) in vars]

B19013_001E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B19013_001E", "ucgid" => "pseudo(0100000US\$1400000)")

B19054_002E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B19054_002E", "ucgid" => "pseudo(0100000US\$1400000)")

B19054_001E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B19054_001E", "ucgid" => "pseudo(0100000US\$1400000)")

B19058_002E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B19058_002E", "ucgid" => "pseudo(0100000US\$1400000)")

B19058_001E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c239d445d96e744bead4b7880", "get" => "NAME,B19058_001E", "ucgid" => "pseudo(0100000US\$1400000)")

B25077_001E
https://api.census.gov/data/2023/acs/acs5
Dict("key" => "0da2f8145563d27c

23-element Vector{DataFrame}:
 85381×3 DataFrame
   Row │ ucgid                 NAME                               B19013_001E 
       │ String                String                             String      
───────┼──────────────────────────────────────────────────────────────────────
     1 │ 1400000US01001020100  Census Tract 201; Autauga County…  62407
     2 │ 1400000US01001020200  Census Tract 202; Autauga County…  59028
     3 │ 1400000US01001020300  Census Tract 203; Autauga County…  66698
     4 │ 1400000US01001020400  Census Tract 204; Autauga County…  75956
     5 │ 1400000US01001020501  Census Tract 205.01; Autauga Cou…  81936
     6 │ 1400000US01001020502  Census Tract 205.02; Autauga Cou…  72998
     7 │ 1400000US01001020503  Census Tract 205.03; Autauga Cou…  88395
     8 │ 1400000US01001020600  Census Tract 206; Autauga County…  75375
     9 │ 1400000US01001020700  Census Tract 207; Autauga County…  62632
    10 │ 1400000US01001020801  Census Tract 208.01; Autauga Cou…  

In [32]:
df = dfs[1]
for next_df in dfs[2:end]
    leftjoin!(df, next_df, on=["NAME", "ucgid"])
end

fname = "raw_data/census/census_data_ndi_tract_raw.csv"
CSV.write(fname, df[!, cols])

LoadError: ArgumentError: column name :MedHHInc not found in the data frame

In [33]:
fname = "raw_data/census/census_data_ndi_tract_raw.csv"
df = DataFrame(CSV.File(fname));

In [34]:
rename!(df, vars);

for (_, name) in vars
    df[!, name] = [fix(e) for e in df[!, name]]
end

LoadError: ArgumentError: Tried renaming :B19013_001E to :MedHHInc, when :B19013_001E does not exist in the data frame.

In [35]:
df[!, "PctRecvIDR"] = 100 * df[!, "PctRecvIDR_num"] ./ df[!, "PctRecvIDR_den"]
df[!, "PctPubAsst"] = 100 * df[!, "PctPubAsst_num"] ./ df[!, "PctPubAsst_den"]
df[!, "PctMgmtBusSciArt"] = 100 * df[!, "PctMgmtBusScArti_num"] ./ df[!, "PctMgmtBusScArti_den"]
df[!, "PctFemHeadKids"] = 100 * (df[!, "PctFemHeadKids_num1"] + df[!, "PctFemHeadKids_num2"]) ./ df[!, "PctFemHeadKids_den"]
df[!, "PctEducHSPlus"] = (df[!, "PctEduc_num25upHS"] .+ df[!, "PctEduc_num25upSC"] .+ df[!, "PctEduc_num25upAD"] \
                       .+ df[!, "PctEduc_num25upBD"] .+ df[!, "PctEduc_num25upGD"]) ./ df[!, "PctEduc_den25up"] * 100
df[!, "PctEducBchPlus"] = 100 * (df[!, "PctEduc_num25upBD"] .+ df[!, "PctEduc_num25upGD"]) ./ df[!, "PctEduc_den25up"]

df[!, "PctRecvIDR"] = 100 * df[!, "PctRecvIDR_num"] ./ df[!, "PctRecvIDR_den"]
df[!, "PctPubAsst"] = 100 * df[!, "PctMgmtBusScArti_num"] ./ df[!, "PctMgmtBusScArti_den"]
df[!, "PctFemHeadKids"] = 100 * (df[!, "PctFemHeadKids_num1"] + df[!, "PctFemHeadKids_num2"]) ./ df[!, "PctFemHeadKids_den"]
df[!, "PctEducHSPlus"] = (df[!, "PctEduc_num25upHS"] .+ df[!, "PctEduc_num25upSC"] .+ df[!, "PctEduc_num25upAD"] \
                       .+ df[!, "PctEduc_num25upBD"] .+ df[!, "PctEduc_num25upGD"]) ./ df[!, "PctEduc_den25up"] * 100
df[!, "PctEducBchPlus"] = 100 * (df[!, "PctEduc_num25upBD"] + df[!, "PctEduc_num25upGD"]) ./ df[!, "PctEduc_den25up"]

df[!, "PctNoIDR"] = 100 .- df[!, "PctRecvIDR"]
df[!, "PctWorkClass"] = 100 .- df[!, "PctMgmtBusSciArt"]
df[!, "PctNotOwnerOcc"] = 100 .- df[!, "PctOwnerOcc"]
df[!, "PctEducLTHS"] = 100 .- df[!, "PctEducHSPlus"]
df[!, "PctEducLTBch"] = 100 .- df[!, "PctEducBchPlus"]

LoadError: ArgumentError: column name :PctRecvIDR_num not found in the data frame

In [36]:
cols = ["ucgid",
        "NAME",
        "MedHHInc",
        "PctNoIDR",
        "PctPubAsst",
        "MedHomeVal",
        "PctWorkClass",
        "PctFemHeadKids",
        "PctNotOwnerOcc",
        "PctNoPhone",
        "PctNComPlmb",
        "PctEducLTHS",
        "PctEducLTBch",
        "PctFamBelowPov",
        "PctUnempl",
        "TotalPopulation"]


fname = "raw_data/census/census_data_ndi_tract.csv"
CSV.write(fname, df[!, cols])

"raw_data/census/census_data_ndi_tract.csv"